In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ROOT_DIR = os.path.dirname(os.path.abspath(""))

file = os.path.join(ROOT_DIR, "data/sitges_access.csv")
df = pd.read_csv(file)
df.head()

,Unnamed: 0,server_name,IP,logname,authenticate,date,petition,URL,status,bytes,referer,user-agent,level
0,0,sitgesanytime.com,47.76.35.19,-,-,2024-01-22 00:00:00+01:00,HEAD,/fr/pag492/explora-platges-i-ports-2/id12/les-...,301,4840,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0
1,1,www.sitgesanytime.com,47.76.35.19,-,-,2024-01-22 00:00:01+01:00,HEAD,/fr/pag492/explora-platges-i-ports-2/id12/les-...,200,5223,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0
2,2,www.sitgesanytime.com,40.77.167.53,-,-,2024-01-22 00:00:06+01:00,GET,/ca/noticias/84/sitges-obt%C3%A9-el-certificat...,404,2509,-,"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Ge...",5
3,3,sitgesanytime.com,47.76.35.19,-,-,2024-01-22 00:00:06+01:00,HEAD,/fr/pag492/explora-platges-i-ports-2/id12/les-...,301,4840,-,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,0
4,4,www.sitgesanytime.com,47.76.35.19,-,-,2024-01-22 00:00:07+01:00,HEAD,/fr/pag492/explora-platges-i-ports-2/id12/les-...,200,5260,-,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,0


In [2]:
df_clean = df.copy()

### Drop useless columns

In [3]:
# remove columns
df_clean = df_clean.drop(columns=["logname", "authenticate", "Unnamed: 0"])
df_clean.columns

Index(['server_name', 'IP', 'date', 'petition', 'URL', 'status', 'bytes',
       'referer', 'user-agent', 'level'],
      dtype='object')

### Server name

### IP

### Date

In [3]:
df_clean["date"] = df["date"].str.split("+").str[0]
df_clean["date"].head()

0    2024-01-22 00:00:00
1    2024-01-22 00:00:01
2    2024-01-22 00:00:06
3    2024-01-22 00:00:06
4    2024-01-22 00:00:07
Name: date, dtype: object

### Petition

In [5]:
df_clean["petition"].value_counts()

petition
GET        1024862
POST         27258
HEAD         24800
-              436
CONNECT         29
OPTIONS          7
USER             1
Name: count, dtype: int64

In [8]:
df_clean = pd.get_dummies(df_clean, columns=["petition"])
columns_to_combine = ["petition_CONNECT", "petition_OPTIONS", "petition_USER"]
df_clean["petition_other"] = df_clean[columns_to_combine].max(axis=1) # combine using OR
df_clean = df_clean.drop(columns=columns_to_combine)
print(df_clean["petition_other"].value_counts())
df_clean.head()

petition_other
False    1077356
True          37
Name: count, dtype: int64


,server_name,IP,date,URL,status,bytes,referer,user-agent,level,petition_-,petition_GET,petition_HEAD,petition_POST,petition_other
0,sitgesanytime.com,47.76.35.19,2024-01-22 00:00:00,/fr/pag492/explora-platges-i-ports-2/id12/les-...,301,4840,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,False,False,True,False,False
1,www.sitgesanytime.com,47.76.35.19,2024-01-22 00:00:01,/fr/pag492/explora-platges-i-ports-2/id12/les-...,200,5223,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,False,False,True,False,False
2,www.sitgesanytime.com,40.77.167.53,2024-01-22 00:00:06,/ca/noticias/84/sitges-obt%C3%A9-el-certificat...,404,2509,-,"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Ge...",5,False,True,False,False,False
3,sitgesanytime.com,47.76.35.19,2024-01-22 00:00:06,/fr/pag492/explora-platges-i-ports-2/id12/les-...,301,4840,-,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,0,False,False,True,False,False
4,www.sitgesanytime.com,47.76.35.19,2024-01-22 00:00:07,/fr/pag492/explora-platges-i-ports-2/id12/les-...,200,5260,-,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,0,False,False,True,False,False


### URL

### Status

### Bytes

### Referer

### User agent